In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题

# 显示多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
#忽略警告
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("./train7.9_screening.csv")
test = pd.read_csv("./test7.9_screening.csv")
df= pd.read_csv("./y_train.csv")

In [3]:
train.drop('Unnamed: 0',axis  = 1,inplace = True)
test.drop('Unnamed: 0',axis  = 1,inplace = True)

In [4]:
label = df['flag']

# 训练模型

In [5]:
from imblearn.over_sampling import SMOTE
model_smote=SMOTE()
train,label=model_smote.fit_resample(train,label)
train.shape,label.shape

((1400000, 70), (1400000,))

In [6]:
# 确定类别特征
cat_features = []
for feature in train.columns:
    if train[feature].nunique() <= 12:
        cat_features.append(feature)
print('类别特征',cat_features)

类别特征 ['settime_month', 'null_Channel2_transaction', 'null_Self_service_transaction']


In [7]:
for i in train.columns:
    if i in cat_features:
        train[i] = train[i].astype('str')
for i in test.columns:
    if i in cat_features:
        test[i] = test[i].astype('str')

In [8]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(loss_function="Logloss",eval_metric="F1",task_type="CPU",od_type="Iter",
                            random_seed=2021,learning_rate=0.07235984828342713, iterations=910, depth=13,l2_leaf_reg=9)
clf.fit(train,label,cat_features=cat_features) 

0:	learn: 0.9631742	total: 2.71s	remaining: 41m 3s
1:	learn: 0.9660301	total: 4.89s	remaining: 37m
2:	learn: 0.9674074	total: 7.2s	remaining: 36m 18s
3:	learn: 0.9678600	total: 9.29s	remaining: 35m 3s
4:	learn: 0.9683024	total: 11.5s	remaining: 34m 47s
5:	learn: 0.9691548	total: 13.6s	remaining: 34m 8s
6:	learn: 0.9696060	total: 15.8s	remaining: 33m 59s
7:	learn: 0.9699874	total: 17.9s	remaining: 33m 37s
8:	learn: 0.9702601	total: 20s	remaining: 33m 22s
9:	learn: 0.9707558	total: 22.1s	remaining: 33m 9s
10:	learn: 0.9714249	total: 24.5s	remaining: 33m 22s
11:	learn: 0.9718071	total: 26.7s	remaining: 33m 20s
12:	learn: 0.9721343	total: 28.9s	remaining: 33m 10s
13:	learn: 0.9725076	total: 31.1s	remaining: 33m 10s
14:	learn: 0.9728963	total: 33.1s	remaining: 32m 55s
15:	learn: 0.9731973	total: 35.6s	remaining: 33m 10s
16:	learn: 0.9735906	total: 38.2s	remaining: 33m 27s
17:	learn: 0.9739752	total: 40.5s	remaining: 33m 27s
18:	learn: 0.9742379	total: 42.8s	remaining: 33m 27s
19:	learn: 0.9

# 输出概率

In [12]:
train = pd.read_csv("./train7.9_screening.csv")
train.drop('Unnamed: 0',axis  = 1,inplace = True)

In [14]:
pro_train=clf.predict_proba(train)[:,-1]
train_cb = pd.DataFrame(pro_train,columns=['flag'])
train_cb.to_csv('./prob_catboost_train.csv',index = False)

In [46]:
test_pred = clf.predict_proba(test)[:,-1]

In [49]:
result_cb = pd.DataFrame(test_pred,columns=['flag'])
result_cb.to_csv('./prob_catboost.csv',index = False)